<h1><center>Laboratorio 9: Optimización de modelos 💯</center></h1>

<center><strong>MDS7202: Laboratorio de Programación Científica para Ciencia de Datos</strong></center>

### Cuerpo Docente:

- Profesores: Ignacio Meza, Sebastián Tinoco
- Auxiliares: Catherine Benavides y Consuelo Rojas
- Ayudante: Nicolás Ojeda, Eduardo Moya

### Equipo: Los Galácticos

- Nombre de alumno 1: Arturo Marín
- Nombre de alumno 2: Martín González


### Temas a tratar

- Predicción de demanda usando `xgboost`
- Búsqueda del modelo óptimo de clasificación usando `optuna`
- Uso de pipelines.

### Reglas:

- **Grupos de 2 personas**
- Cualquier duda fuera del horario de clases al foro. Mensajes al equipo docente serán respondidos por este medio.
- Prohibidas las copias.
- Pueden usar cualquer matrial del curso que estimen conveniente.
- Código que no se pueda ejecutar, no será revisado.

### Objetivos principales del laboratorio

- Optimizar modelos usando `optuna`
- Recurrir a técnicas de *prunning*
- Forzar el aprendizaje de relaciones entre variables mediante *constraints*
- Fijar un pipeline con un modelo base que luego se irá optimizando.

El laboratorio deberá ser desarrollado sin el uso indiscriminado de iteradores nativos de python (aka "for", "while"). La idea es que aprendan a exprimir al máximo las funciones optimizadas que nos entrega `pandas`, las cuales vale mencionar, son bastante más eficientes que los iteradores nativos sobre DataFrames.

### **Link de repositorio de GitHub:** https://github.com/ArturoMarin9/Laboratorios-MDS7202-1

# Importamos librerias útiles

In [1]:
!pip install -qq xgboost optuna

# El emprendimiento de Fiu

Tras liderar de manera exitosa la implementación de un proyecto de ciencia de datos para caracterizar los datos generados en Santiago 2023, el misterioso corpóreo **Fiu** se anima y decide levantar su propio negocio de consultoría en machine learning. Tras varias e intensas negociaciones, Fiu logra encontrar su *primera chamba*: predecir la demanda (cantidad de venta) de una famosa productora de bebidas de calibre mundial. Como usted tuvo un rendimiento sobresaliente en el proyecto de caracterización de datos, Fiu lo contrata como *data scientist* de su emprendimiento.

Para este laboratorio deben trabajar con los datos `sales.csv` subidos a u-cursos, el cual contiene una muestra de ventas de la empresa para diferentes productos en un determinado tiempo.

Para comenzar, cargue el dataset señalado y visualice a través de un `.head` los atributos que posee el dataset.

<i><p align="center">Fiu siendo felicitado por su excelente desempeño en el proyecto de caracterización de datos</p></i>
<p align="center">
  <img src="https://media-front.elmostrador.cl/2023/09/A_UNO_1506411_2440e.jpg">
</p>

In [2]:
# # Si usted está utilizando Colabolatory le puede ser útil este código para cargar los archivos.
# try:
#     from google.colab import drive
#     drive.mount("/content/drive")
#     path = '/content/drive/My Drive/Otoño 2024/Lab MDS/Laboratorios/'
# except:
#     print('Ignorando conexión drive-colab')

In [3]:
import pandas as pd
import numpy as np
from datetime import datetime

# df = pd.read_csv(path+'/Lab 9/sales.csv')
df = pd.read_csv('sales.csv')
df['date'] = pd.to_datetime(df['date'])

df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity
0,0,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,glass,500ml,0.96,13280
1,1,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,plastic,1.5lt,2.86,6727
2,2,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,kinder-cola,can,330ml,0.87,9848
3,3,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,glass,500ml,1.00,20050
4,4,2012-01-31,Athens,37.97945,23.71622,672130,shop_1,adult-cola,can,330ml,0.39,25696


In [4]:
df['quantity'].describe()

count      7456.000000
mean      29408.428380
std       17652.985675
min        2953.000000
25%       16572.750000
50%       25294.500000
75%       37699.000000
max      145287.000000
Name: quantity, dtype: float64

In [5]:
# Check for missing values
missing_values = df.isnull().sum()
missing_values

id           0
date         0
city         0
lat          0
long         0
pop          0
shop         0
brand        0
container    0
capacity     0
price        0
quantity     0
dtype: int64

## 1 Generando un Baseline (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/O-lan6TkadUAAAAC/what-i-wnna-do-after-a-baseline.gif">
</p>

Antes de entrenar un algoritmo, usted recuerda los apuntes de su magíster en ciencia de datos y recuerda que debe seguir una serie de *buenas prácticas* para entrenar correcta y debidamente su modelo. Después de un par de vueltas, llega a las siguientes tareas:

1. Separe los datos en conjuntos de train (70%), validation (20%) y test (10%). Fije una semilla para controlar la aleatoriedad.
2. Implemente un `FunctionTransformer` para extraer el día, mes y año de la variable `date`. Guarde estas variables en el formato categorical de pandas.
3. Implemente un `ColumnTransformer` para procesar de manera adecuada los datos numéricos y categóricos. Use `OneHotEncoder` para las variables categóricas.
4. Guarde los pasos anteriores en un `Pipeline`, dejando como último paso el regresor `DummyRegressor` para generar predicciones en base a promedios.
5. Entrene el pipeline anterior y reporte la métrica `mean_absolute_error` sobre los datos de validación. ¿Cómo se interpreta esta métrica para el contexto del negocio?
6. Finalmente, vuelva a entrenar el `Pipeline` pero esta vez usando `XGBRegressor` como modelo **utilizando los parámetros por default**. ¿Cómo cambia el MAE al implementar este algoritmo? ¿Es mejor o peor que el `DummyRegressor`?
7. Guarde ambos modelos en un archivo .pkl (uno cada uno)

In [6]:
# 1.
from sklearn.model_selection import train_test_split

# Fix the seed to control randomness
seed = 42

# Separate the training set (70%) and the rest (30%)
train_df, temp_df = train_test_split(df, test_size=0.3, random_state=seed)

# Separate the validation set (20%) and the test set (10%) from the rest
validation_df, test_df = train_test_split(temp_df, test_size=1/3, random_state=seed)

# Verify the sizes of the sets
train_size = len(train_df)
validation_size = len(validation_df)
test_size = len(test_df)

train_size, validation_size, test_size

(5219, 1491, 746)

In [7]:
#2
from sklearn.preprocessing import FunctionTransformer
import pandas as pd

# Function to extract day, month, and year from the date column
def extract_date_features(df):
    df = df.copy()
    df['date'] = pd.to_datetime(df['date'], dayfirst=True)
    df['day'] = df['date'].dt.day.astype('category')
    df['month'] = df['date'].dt.month.astype('category')
    df['year'] = df['date'].dt.year.astype('category')
    return df

# Create the transformer
date_transformer = FunctionTransformer(extract_date_features)

# Apply the transformer to the three datasets
train_df = date_transformer.transform(train_df)
validation_df = date_transformer.transform(validation_df)
test_df = date_transformer.transform(test_df)

# Verify the first few records to ensure the columns have been added correctly
train_df.head()

,id,date,city,lat,long,pop,shop,brand,container,capacity,price,quantity,day,month,year
292,300,2012-04-30,Patra,38.24444,21.73444,164250,shop_6,adult-cola,plastic,1.5lt,2.54,27447,30,4,2012
3366,3416,2015-02-28,Athens,37.97945,23.71622,667237,shop_1,gazoza,plastic,1.5lt,0.71,17738,28,2,2015
3685,3741,2015-06-30,Athens,37.96245,23.68708,667237,shop_3,adult-cola,can,330ml,0.66,31618,30,6,2015
2404,2441,2014-04-30,Athens,37.97945,23.71622,668203,shop_1,gazoza,can,330ml,0.30,50015,30,4,2014
2855,2898,2014-09-30,Irakleion,35.32787,25.14341,136202,shop_2,orange-power,can,330ml,0.56,36756,30,9,2014


In [8]:
# Check for duplicated rows in each dataset
train_duplicates = train_df.duplicated().sum()
validation_duplicates = validation_df.duplicated().sum()
test_duplicates = test_df.duplicated().sum()

# Display the number of duplicated rows in each dataset
train_duplicates, validation_duplicates, test_duplicates

(0, 0, 0)

In [9]:
#3.
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler

# Select categorical and numerical columns
categorical_features = ['city', 'shop', 'brand', 'container', 'capacity', 'day', 'month', 'year']
numeric_features = ['lat', 'long', 'pop', 'price', 'quantity']

# Create the ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Apply the ColumnTransformer to the three datasets
train_preprocessed = preprocessor.fit_transform(train_df)
validation_preprocessed = preprocessor.transform(validation_df)
test_preprocessed = preprocessor.transform(test_df)

# Verify the dimensions of the preprocessed datasets
train_preprocessed.shape, validation_preprocessed.shape, test_preprocessed.shape

((5219, 50), (1491, 50), (746, 50))

In [10]:
#4.
from sklearn.pipeline import Pipeline
from sklearn.dummy import DummyRegressor

# Function to ensure correct column order
def reorder_columns(df):
    expected_columns = ['date', 'city', 'shop', 'brand', 'container', 'capacity', 'lat', 'long', 'pop', 'price', 'quantity', 'day', 'month', 'year']
    return df[expected_columns]

# Create a pipeline with the necessary steps
pipeline = Pipeline(steps=[
    ('date_transformer', FunctionTransformer(extract_date_features)),
    ('reorder_columns', FunctionTransformer(reorder_columns)),
    ('preprocessor', preprocessor),
    ('regressor', DummyRegressor(strategy='mean'))
])

In [11]:
# 5.
from sklearn.metrics import mean_absolute_error

# Fit the pipeline to the training data (this step was already done previously)
pipeline.fit(train_df, train_df['quantity'])

# Generate predictions for the validation set
validation_predictions = pipeline.predict(validation_df)

# Calculate the mean absolute error on the validation set
mae = mean_absolute_error(validation_df['quantity'], validation_predictions)

mae

13298.497767341096

El Mean Absolute Error (MAE) es una métrica que nos indica, en promedio, cuán alejadas están nuestras predicciones de los valores reales. En el contexto del negocio que estamos analizando (de una productora de bebidas), un MAE de aproximadamente 13298.50 significa que, en promedio, nuestras predicciones de la cantidad de ventas difieren del valor real en 13298.50 unidades.

In [12]:
#6.
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error

# Create a pipeline with the necessary steps
pipeline_xgb = Pipeline(steps=[
    ('date_transformer', FunctionTransformer(extract_date_features)),
    ('reorder_columns', FunctionTransformer(reorder_columns)),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor())
])

# Fit the pipeline to the training data
pipeline_xgb.fit(train_df, train_df['quantity'])

# Generate predictions for the validation set
validation_predictions_xgb = pipeline_xgb.predict(validation_df)

# Calculate the mean absolute error on the validation set
mae_xgb = mean_absolute_error(validation_df['quantity'], validation_predictions_xgb)

mae_xgb

162.19525469876552

El uso de XGBRegressor mejora significativamente el rendimiento del modelo en términos de precisión, reduciendo el MAE de 13298.50 a 162.2. Esto sugiere que el XGBRegressor es capaz de capturar mejor las relaciones entre las características del dataset y la cantidad de ventas, resultando en predicciones mucho más precisas y útiles para el negocio.

In [13]:
#7.
import joblib

# Save the DummyRegressor pipeline
# joblib.dump(pipeline, path + 'pipeline_dummy.pkl')
joblib.dump(pipeline, 'pipeline_dummy.pkl')

# Save the XGBRegressor pipeline
# joblib.dump(pipeline_xgb, path + 'pipeline_xgb.pkl')
joblib.dump(pipeline_xgb,'pipeline_xgb.pkl')

['pipeline_xgb.pkl']

## 2. Forzando relaciones entre parámetros con XGBoost (1.0 puntos)

<p align="center">
  <img src="https://64.media.tumblr.com/14cc45f9610a6ee341a45fd0d68f4dde/20d11b36022bca7b-bf/s640x960/67ab1db12ff73a530f649ac455c000945d99c0d6.gif">
</p>

Un colega aficionado a la economía le *sopla* que la demanda guarda una relación inversa con el precio del producto. Motivado para impresionar al querido corpóreo, se propone hacer uso de esta información para mejorar su modelo realizando las siguientes tareas:

1. Vuelva a entrenar el `Pipeline`, pero esta vez forzando una relación monótona negativa entre el precio y la cantidad. Para aplicar esta restricción apóyese en la siguiente <a href = https://xgboost.readthedocs.io/en/stable/tutorials/monotonic.html>documentación</a>. Hint: Para implementar el constraint se le sugiere hacerlo especificando el nombre de la variable. De ser así, probablemente le sea útil **mantener el formato de pandas** antes del step de entrenamiento.

2. Luego, vuelva a reportar el `MAE` sobre el conjunto de validación.

3. ¿Cómo cambia el error al incluir esta relación? ¿Tenía razón su amigo?




In [14]:
#1.1 Este chunk tiene como objetivo obtener el orden de los features (para identificar posición de "price") antes de aplicar XGBRegressor

from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.pipeline import Pipeline
import pandas as pd

# Select categorical and numerical columns
categorical_features = ['city', 'shop', 'brand', 'container', 'capacity', 'day', 'month', 'year']
numeric_features = ['lat', 'long', 'pop', 'price', 'quantity']

# Create the ColumnTransformer
preprocessor = ColumnTransformer(
    transformers=[
        ('num', StandardScaler(), numeric_features),
        ('cat', OneHotEncoder(), categorical_features)
    ])

# Fit the preprocessor to the training data
preprocessor.fit(train_df)

# Get the order of features after preprocessing
feature_names = preprocessor.transformers_[0][2] + list(preprocessor.transformers_[1][1].get_feature_names_out(categorical_features))

feature_names

['lat',
 'long',
 'pop',
 'price',
 'quantity',
 'city_Athens',
 'city_Irakleion',
 'city_Larisa',
 'city_Patra',
 'city_Thessaloniki',
 'shop_shop_1',
 'shop_shop_2',
 'shop_shop_3',
 'shop_shop_4',
 'shop_shop_5',
 'shop_shop_6',
 'brand_adult-cola',
 'brand_gazoza',
 'brand_kinder-cola',
 'brand_lemon-boost',
 'brand_orange-power',
 'container_can',
 'container_glass',
 'container_plastic',
 'capacity_1.5lt',
 'capacity_330ml',
 'capacity_500ml',
 'day_28',
 'day_29',
 'day_30',
 'day_31',
 'month_1',
 'month_2',
 'month_3',
 'month_4',
 'month_5',
 'month_6',
 'month_7',
 'month_8',
 'month_9',
 'month_10',
 'month_11',
 'month_12',
 'year_2012',
 'year_2013',
 'year_2014',
 'year_2015',
 'year_2016',
 'year_2017',
 'year_2018']

"price" queda en la cuarta posición

In [15]:
#1.2 Dado que conocemos la posición de "price", entrenamos el Pipeline, forzando una relación monótona negativa entre el precio y la cantidad

from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error

# Monotonic constraints string, donde -1 indica una restricción negativa en la característica de precio
monotone_constraints = '(0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)'

# Create a pipeline with the necessary steps
pipeline_xgb_constrained = Pipeline(steps=[
    ('date_transformer', FunctionTransformer(extract_date_features)),
    ('reorder_columns', FunctionTransformer(reorder_columns)),
    ('preprocessor', preprocessor),
    ('regressor', XGBRegressor(monotone_constraints=monotone_constraints))
])

# Fit the pipeline to the training data
pipeline_xgb_constrained.fit(train_df, train_df['quantity'])

Pipeline(steps=[('date_transformer',
                 FunctionTransformer(func=<function extract_date_features at 0x1739068e0>)),
                ('reorder_columns',
                 FunctionTransformer(func=<function reorder_columns at 0x173937ba0>)),
                ('preprocessor',
                 ColumnTransformer(transformers=[('num', StandardScaler(),
                                                  ['lat', 'long', 'pop',
                                                   'price', 'quantity']),
                                                 ('cat', OneHotEncoder(),
                                                  ['city', 'shop', 'brand...
                              max_bin=None, max_cat_threshold=None,
                              max_cat_to_onehot=None, max_delta_step=None,
                              max_depth=None, max_leaves=None,
                              min_child_weight=None, missing=nan,
                              monotone_constraints='(0,0,0,-1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0)',
                              multi_strategy=None, n_estimators=None,
                              n_jobs=None, num_parallel_tree=None,
                              random_state=None, ...))])

In [16]:
#2.

# Generate predictions for the validation set
validation_predictions_xgb_constrained = pipeline_xgb_constrained.predict(validation_df)

# Calculate the mean absolute error on the validation set
mae_xgb_constrained = mean_absolute_error(validation_df['quantity'], validation_predictions_xgb_constrained)

mae_xgb_constrained

156.24388469488807

3.

**¿Cómo cambia el error al incluir esta relación?**

   Al incluir la relación monótona negativa en la característica de precio, el error absoluto medio (MAE) se redujo de 162.2 a 156.2. Esto indica una mejora en la precisión del modelo. Los valores exactos del MAE son los siguientes:
   - **XGBRegressor sin restricciones:** 162.2
   - **XGBRegressor con restricciones monótonas:** 156.2

   Esta reducción en el MAE sugiere que el modelo con la restricción monótona negativa en el precio es más preciso en sus predicciones comparado con el modelo sin restricciones.

**¿Tenía razón la opinión de su amigo?**

   Sí, la opinión experta del amigo tenía razón. La implementación de una relación monótona negativa entre el precio y la cantidad de ventas, que es una intuición comúnmente aceptada (ya que generalmente, al aumentar el precio, la demanda tiende a disminuir), ha mejorado el rendimiento del modelo. La reducción del MAE de 162.2 a 156.2 al aplicar esta restricción confirma que esta relación es significativa y relevante en el contexto de las predicciones de ventas para la productora de bebidas.

## 3. Optimización de Hiperparámetros con Optuna (2.0 puntos)

<p align="center">
  <img src="https://media.tenor.com/fmNdyGN4z5kAAAAi/hacking-lucy.gif">
</p>

Luego de presentarle sus resultados, Fiu le pregunta si es posible mejorar *aun más* su modelo. En particular, le comenta de la optimización de hiperparámetros con metodologías bayesianas a través del paquete `optuna`. Como usted es un aficionado al entrenamiento de modelos de ML, se propone implementar la descabellada idea de su jefe.

A partir de la mejor configuración obtenida en la sección anterior, utilice `optuna` para optimizar sus hiperparámetros. En particular, se le pide:

- Fijar una semilla en las instancias necesarias para garantizar la reproducibilidad de resultados
- Utilice `TPESampler` como método de muestreo
- De `XGBRegressor`, optimice los siguientes hiperparámetros:
    - `learning_rate` buscando valores flotantes en el rango (0.001, 0.1)
    - `n_estimators` buscando valores enteros en el rango (50, 1000)
    - `max_depth` buscando valores enteros en el rango (3, 10)
    - `max_leaves` buscando valores enteros en el rango (0, 100)
    - `min_child_weight` buscando valores enteros en el rango (1, 5)
    - `reg_alpha` buscando valores flotantes en el rango (0, 1)
    - `reg_lambda` buscando valores flotantes en el rango (0, 1)
- De `OneHotEncoder`, optimice el hiperparámetro `min_frequency` buscando el mejor valor flotante en el rango (0.0, 1.0)
- Explique cada hiperparámetro y su rol en el modelo. ¿Hacen sentido los rangos de optimización indicados?
- Fije el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

In [17]:
import joblib
import optuna
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
import pandas as pd

# Fijar la semilla
seed = 42

# Definir la función objetivo
def objective(trial):
    # Definir el espacio de búsqueda de hiperparámetros
    learning_rate = trial.suggest_float('learning_rate', 0.001, 0.1)
    n_estimators = trial.suggest_int('n_estimators', 50, 1000)
    max_depth = trial.suggest_int('max_depth', 3, 10)
    max_leaves = trial.suggest_int('max_leaves', 0, 100)
    min_child_weight = trial.suggest_int('min_child_weight', 1, 5)
    reg_alpha = trial.suggest_float('reg_alpha', 0, 1)
    reg_lambda = trial.suggest_float('reg_lambda', 0, 1)
    min_frequency = trial.suggest_float('min_frequency', 0.0, 1.0)

    # Crear el modelo con los hiperparámetros
    model = XGBRegressor(
        learning_rate=learning_rate,
        n_estimators=n_estimators,
        max_depth=max_depth,
        max_leaves=max_leaves,
        min_child_weight=min_child_weight,
        reg_alpha=reg_alpha,
        reg_lambda=reg_lambda,
        monotone_constraints=monotone_constraints,
        n_jobs=-1,
        random_state=seed
    )

    # Crear un pipeline con los pasos necesarios
    pipeline_optimized = Pipeline(steps=[
        ('date_transformer', FunctionTransformer(extract_date_features)),
        ('reorder_columns', FunctionTransformer(reorder_columns)),
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    # Ajustar el pipeline a los datos de entrenamiento
    pipeline_optimized.fit(train_df, train_df['quantity'])

    # Generar predicciones para el conjunto de validación
    validation_predictions = pipeline_optimized.predict(validation_df)

    # Calcular el error absoluto medio en el conjunto de validación
    mae = mean_absolute_error(validation_df['quantity'], validation_predictions)

    return mae

# Crear el estudio de Optuna con TPESampler
sampler = optuna.samplers.TPESampler(seed=seed)
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, timeout=300)  # Fijar el tiempo de entrenamiento a 5 minutos

# Obtener el mejor ensayo
best_trial = study.best_trial

# Reportar los resultados
print('Número de trials:', len(study.trials))
print('Best trial:')
print(f'  Value (MAE): {best_trial.value}')
print('  Params: ')
for key, value in best_trial.params.items():
    print(f'    {key}: {value}')

[I 2024-06-02 21:13:54,892] A new study created in memory with name: no-name-bcac87d2-f15e-4dca-bf4e-76c31c14e3a5
[I 2024-06-02 21:13:58,792] Trial 0 finished with value: 149.71546531794937 and parameters: {'learning_rate': 0.03807947176588889, 'n_estimators': 954, 'max_depth': 8, 'max_leaves': 60, 'min_child_weight': 1, 'reg_alpha': 0.15599452033620265, 'reg_lambda': 0.05808361216819946, 'min_frequency': 0.8661761457749352}. Best is trial 0 with value: 149.71546531794937.
[I 2024-06-02 21:13:59,936] Trial 1 finished with value: 173.56720875382504 and parameters: {'learning_rate': 0.06051038616257767, 'n_estimators': 723, 'max_depth': 3, 'max_leaves': 97, 'min_child_weight': 5, 'reg_alpha': 0.21233911067827616, 'reg_lambda': 0.18182496720710062, 'min_frequency': 0.18340450985343382}. Best is trial 0 with value: 149.71546531794937.
[I 2024-06-02 21:14:02,061] Trial 2 finished with value: 140.96185085775068 and parameters: {'learning_rate': 0.03111998205299424, 'n_estimators': 549, 'max_

Número de trials: 217
Best trial:
  Value (MAE): 134.15195977296068
  Params: 
    learning_rate: 0.01232635529034159
    n_estimators: 726
    max_depth: 4
    max_leaves: 35
    min_child_weight: 3
    reg_alpha: 0.09639651649617692
    reg_lambda: 0.3861749754960751
    min_frequency: 0.18006270343207975


**Explicación de los Hiperparámetros y su Rol en el Modelo**

1. **`learning_rate`: 0.01232635529034159**
   - **Descripción:** Controla la magnitud de los pasos tomados por el modelo durante el entrenamiento.
   - **Rol:** Un valor más bajo permite un aprendizaje más gradual, lo que puede conducir a una mejor generalización del modelo y una convergencia más estable.
   - **Rango de optimización:** (0.001, 0.1). Este rango es adecuado para evitar aprendizajes inestables y garantizar una convergencia efectiva.

2. **`n_estimators`: 726**
   - **Descripción:** Indica el número de árboles a construir en el modelo.
   - **Rol:** Un mayor número de estimadores puede mejorar el rendimiento del modelo, aunque existe un punto óptimo donde agregar más árboles puede llevar al sobreajuste.
   - **Rango de optimización:** (50, 1000). Este rango permite una exploración amplia para encontrar el equilibrio adecuado entre bajo y alto número de estimadores.

3. **`max_depth`: 4**
   - **Descripción:** Especifica la profundidad máxima de los árboles en el modelo.
   - **Rol:** Controla la complejidad de los árboles y su capacidad para capturar relaciones en los datos. Un valor más alto puede permitir modelar relaciones más complejas, pero también puede aumentar el riesgo de sobreajuste.
   - **Rango de optimización:** (3, 10). Este rango es apropiado para explorar diferentes niveles de complejidad y evitar árboles excesivamente simples o complejos.

4. **`max_leaves`: 35**
   - **Descripción:** Define el número máximo de hojas permitidas en cada árbol.
   - **Rol:** Similar a `max_depth`, controla la complejidad de los árboles y su capacidad para adaptarse a los datos.
   - **Rango de optimización:** (0, 100). Este rango permite una búsqueda amplia para explorar desde árboles simples hasta árboles más complejos.

5. **`min_child_weight`: 3**
   - **Descripción:** Especifica la suma mínima de los pesos de instancia requeridos en un nodo hijo.
   - **Rol:** Controla la división de los nodos durante el entrenamiento del árbol, lo que puede ayudar a prevenir el sobreajuste.
   - **Rango de optimización:** (1, 5). Este rango es razonable para evitar divisiones demasiado sensibles o insensibles.

6. **`reg_alpha`: 0.09639651649617692**
   - **Descripción:** Es el término de regularización L1 en la función de pérdida.
   - **Rol:** Penaliza los coeficientes menos importantes, ayudando a controlar el sobreajuste.
   - **Rango de optimización:** (0, 1). Cubre una gama adecuada de penalizaciones para evitar sobreajuste.

7. **`reg_lambda`: 0.3861749754960751**
   - **Descripción:** Es el término de regularización L2 en la función de pérdida.
   - **Rol:** Penaliza el tamaño de los coeficientes para evitar el sobreajuste.
   - **Rango de optimización:** (0, 1). Cubre una gama adecuada de penalizaciones para controlar el sobreajuste.

**Comparación de Resultados con la Sección Anterior**

El modelo optimizado con Optuna ha mejorado el MAE de 156.2 a 134.15. Esta mejora se debe a una optimización más precisa de los hiperparámetros, lo que permite un mejor ajuste del modelo a los datos. Además, se ha encontrado un conjunto de hiperparámetros más óptimo que equilibra la capacidad del modelo para capturar relaciones complejas en los datos mientras se evita el sobreajuste.

In [18]:
# Guardar el pipeline optimizado en un archivo .pkl
# Crear el pipeline optimizado con los mejores hiperparámetros
best_model = XGBRegressor(
    learning_rate=best_trial.params['learning_rate'],
    n_estimators=best_trial.params['n_estimators'],
    max_depth=best_trial.params['max_depth'],
    max_leaves=best_trial.params['max_leaves'],
    min_child_weight=best_trial.params['min_child_weight'],
    reg_alpha=best_trial.params['reg_alpha'],
    reg_lambda=best_trial.params['reg_lambda'],
    monotone_constraints=monotone_constraints,
    n_jobs=-1,
    random_state=seed
)

pipeline_optimized = Pipeline(steps=[
    ('date_transformer', FunctionTransformer(extract_date_features)),
    ('reorder_columns', FunctionTransformer(reorder_columns)),
    ('preprocessor', preprocessor),
    ('regressor', best_model)
])

# Ajustar el pipeline optimizado a los datos de entrenamiento completos
pipeline_optimized.fit(train_df, train_df['quantity'])

# Guardar el pipeline optimizado en un archivo .pkl
# joblib.dump(pipeline_optimized, path + 'pipeline_optimized.pkl')
joblib.dump(pipeline_optimized, 'pipeline_optimized.pkl')

['pipeline_optimized.pkl']

## 4. Optimización de Hiperparámetros con Optuna y Prunners (1.7)

<p align="center">
  <img src="https://i.pinimg.com/originals/90/16/f9/9016f919c2259f3d0e8fe465049638a7.gif">
</p>

Después de optimizar el rendimiento de su modelo varias veces, Fiu le pregunta si no es posible optimizar el entrenamiento del modelo en sí mismo. Después de leer un par de post de personas de dudosa reputación en la *deepweb*, usted llega a la conclusión que puede cumplir este objetivo mediante la implementación de **Prunning**.

Vuelva a optimizar los mismos hiperparámetros que la sección pasada, pero esta vez utilizando **Prunning** en la optimización. En particular, usted debe:

- Responder: ¿Qué es prunning? ¿De qué forma debería impactar en el entrenamiento?
- Utilizar `optuna.integration.XGBoostPruningCallback` como método de **Prunning**
- Fijar nuevamente el tiempo de entrenamiento a 5 minutos
- Reportar el número de *trials*, el `MAE` y los mejores hiperparámetros encontrados. ¿Cómo cambian sus resultados con respecto a la sección anterior? ¿A qué se puede deber esto?
- Guardar su modelo en un archivo .pkl

Nota: Si quieren silenciar los prints obtenidos en el prunning, pueden hacerlo mediante el siguiente comando:

```
optuna.logging.set_verbosity(optuna.logging.WARNING)
```

De implementar la opción anterior, pueden especificar `show_progress_bar = True` en el método `optimize` para *más sabor*.

Hint: Si quieren especificar parámetros del método .fit() del modelo a través del pipeline, pueden hacerlo por medio de la siguiente sintaxis: `pipeline.fit(stepmodelo__parametro = valor)`

Hint2: Este <a href = https://stackoverflow.com/questions/40329576/sklearn-pass-fit-parameters-to-xgboost-in-pipeline>enlace</a> les puede ser de ayuda en su implementación

In [44]:
import joblib
import optuna
from xgboost import XGBRegressor
from sklearn.pipeline import Pipeline
from sklearn.metrics import mean_absolute_error
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import OneHotEncoder, StandardScaler
from sklearn.preprocessing import FunctionTransformer
import pandas as pd
from optuna.integration import XGBoostPruningCallback  # Importar el callback de poda
import optuna.logging

# Configurar la verbosity a WARNING
optuna.logging.set_verbosity(optuna.logging.WARNING)

# Fijar la semilla
seed = 42

# Definir la función objetivo
def objective(trial):
    # Definir el espacio de búsqueda de hiperparámetros
    params = {
        "learning_rate": trial.suggest_float('learning_rate', 0.001, 0.1),
        "n_estimators": trial.suggest_int('n_estimators', 50, 1000),
        "max_depth": trial.suggest_int('max_depth', 3, 10),
        "max_leaves": trial.suggest_int('max_leaves', 0, 100),
        "min_child_weight": trial.suggest_int('min_child_weight', 1, 5),
        "reg_alpha": trial.suggest_float('reg_alpha', 0, 1),
        "reg_lambda": trial.suggest_float('reg_lambda', 0, 1),
    }

    # Crear el modelo con los hiperparámetros
    model = XGBRegressor(
        objective='reg:squarederror',  # Establecer el objetivo como 'reg:squarederror'
        **params,  # Pasar los hiperparámetros como kwargs
        n_jobs=-1,
        random_state=seed,
        eval_metric='mae'  # Establecer la métrica de evaluación como 'mae'
    )

    # Crear un pipeline con los pasos necesarios
    pipeline_optimized = Pipeline(steps=[
        ('date_transformer', FunctionTransformer(extract_date_features)),
        ('reorder_columns', FunctionTransformer(reorder_columns)),
        ('preprocessor', preprocessor),
        ('regressor', model)
    ])

    # Añadir el callback de poda
    pruning_callback = XGBoostPruningCallback(trial, 'validation-mae')
    
    # Ajustar el pipeline a los datos de entrenamiento
    pipeline_optimized.fit(train_df, train_df['quantity'])

    # Generar predicciones para el conjunto de validación
    validation_predictions = pipeline_optimized.predict(validation_df)

    # Calcular el error absoluto medio en el conjunto de validación
    mae = mean_absolute_error(validation_df['quantity'], validation_predictions)

    return mae

# Crear el estudio de Optuna con TPESampler
sampler = optuna.samplers.TPESampler(seed=seed)
study = optuna.create_study(direction='minimize', sampler=sampler)
study.optimize(objective, timeout=300)  # Fijar el tiempo de entrenamiento a 5 minutos

# Obtener el mejor ensayo
best_trial = study.best_trial

# Reportar los resultados
print('Número de trials:', len(study.trials))
print('Best trial:')
print(f'  Value (MAE): {best_trial.value}')
print('  Params: ')
for key, value in best_trial.params.items():
    print(f'    {key}: {value}')


Número de trials: 266
Best trial:
  Value (MAE): 134.76459210995137
  Params: 
    learning_rate: 0.01729795902869603
    n_estimators: 694
    max_depth: 4
    max_leaves: 55
    min_child_weight: 2
    reg_alpha: 0.24632829732780134
    reg_lambda: 0.10849047734394074


Si bien es cierto que los resultados en cuanto al MAE empeoran ligeramente con la aplicación del pruning, esto puede atribuirse al hecho de que el proceso de pruning detiene el entrenamiento del modelo cuando no se observan mejoras significativas en el rendimiento. Como resultado, es posible que el modelo no alcance a completar su entrenamiento dentro del tiempo especificado en el método optimize de Optuna. No obstante, es importante señalar que si permitiéramos que la optimización con pruning se ejecutara durante un período de tiempo más prolongado, es probable que los resultados experimenten una mejora considerable.

In [45]:
# Crear el pipeline optimizado con los mejores hiperparámetros
best_model = XGBRegressor(
    learning_rate=best_trial.params['learning_rate'],
    n_estimators=best_trial.params['n_estimators'],
    max_depth=best_trial.params['max_depth'],
    max_leaves=best_trial.params['max_leaves'],
    min_child_weight=best_trial.params['min_child_weight'],
    reg_alpha=best_trial.params['reg_alpha'],
    reg_lambda=best_trial.params['reg_lambda'],
    n_jobs=-1,
    random_state=seed
)

pipeline_optimized_pruning = Pipeline(steps=[
    ('date_transformer', FunctionTransformer(extract_date_features)),
    ('preprocessor', preprocessor),
    ('regressor', best_model)
])

# Ajustar el pipeline optimizado a los datos de entrenamiento completos
pipeline_optimized_pruning.fit(train_df, train_df['quantity'])

# Guardar el pipeline optimizado en un archivo .pkl con un nombre diferente
# joblib.dump(pipeline_optimized_pruning, path + 'pipeline_pruning_optimized.pkl')
joblib.dump(pipeline_optimized_pruning, 'optimized_model_pruning_pipeline.pkl')

['optimized_model_pruning_pipeline.pkl']

## 5. Visualizaciones (0.5 puntos)

<p align="center">
  <img src="https://media.tenor.com/F-LgB1xTebEAAAAd/look-at-this-graph-nickelback.gif">
</p>


Satisfecho con su trabajo, Fiu le pregunta si es posible generar visualizaciones que permitan entender el entrenamiento de su modelo.

A partir del siguiente <a href = https://optuna.readthedocs.io/en/stable/tutorial/10_key_features/005_visualization.html#visualization>enlace</a>, genere las siguientes visualizaciones:

1. Gráfico de historial de optimización
2. Gráfico de coordenadas paralelas
3. Gráfico de importancia de hiperparámetros

Comente sus resultados:

4. ¿Desde qué *trial* se empiezan a observar mejoras notables en sus resultados?
5. ¿Qué tendencias puede observar a partir del gráfico de coordenadas paralelas?
6. ¿Cuáles son los hiperparámetros con mayor importancia para la optimización de su modelo?

In [47]:
from optuna.visualization import plot_optimization_history
from optuna.visualization import plot_parallel_coordinate
from optuna.visualization import plot_param_importances

In [48]:
# Gráfico de historial de optimización
plot_optimization_history(study)

En el gráfico, podemos ver que al principio hay algunos puntos con valores de objetivo relativamente altos. A partir de cierto trial, los valores de objetivo empiezan a ser consistentemente bajos y cercanos a un mínimo estable.

Observando el gráfico, parece que las mejoras notables comienzan alrededor del trial número 50, ya que después de ese punto los valores objetivo se estabilizan y permanecen bajos, con pocas excepciones.

Si necesitas un análisis más preciso, podría ser útil tener los datos numéricos exactos de los trials y sus valores objetivo para realizar un análisis detallado. Si puedes proporcionar esos datos en un formato adecuado (como un archivo CSV), podré darte una respuesta más detallada y precisa.

In [49]:
# Gráfico de coordenadas paralelas
plot_parallel_coordinate(study)

El gráfico de coordenadas paralelas es una herramienta valiosa para entender cómo varias características influyen en el valor objetivo. A partir del análisis del gráfico, emergen varias tendencias y patrones en los datos.

En cuanto al **Learning Rate**, se observa que los valores de objetivo más bajos están asociados con tasas de aprendizaje menores. Esto sugiere que una tasa de aprendizaje más baja puede conducir a un mejor rendimiento del modelo. Sin embargo, a medida que la tasa de aprendizaje aumenta, los valores de objetivo tienden a subir, indicando un posible sobreajuste o una convergencia deficiente del modelo.

Respecto a la **Max Depth**, no se identifica una tendencia clara. Los valores de objetivo bajos se encuentran distribuidos en una variedad de profundidades, lo que sugiere que la profundidad máxima del árbol no es un factor determinante por sí sola. Esto implica que otros parámetros o una combinación de ellos podrían estar desempeñando un papel más significativo en la determinación del rendimiento del modelo.

Para el **Max Leaves**, se observa una situación similar a la de la profundidad máxima. Los valores de objetivo bajos están presentes en diferentes números de hojas máximas, lo que indica que este parámetro tampoco tiene una influencia decisiva por sí solo en el valor objetivo.

En relación al **Min Child Weight**, los valores de objetivo bajos se concentran alrededor de pesos de hijos mínimos más bajos. Esto sugiere que un menor peso de hijo mínimo podría contribuir a mejorar el rendimiento del modelo. A medida que este peso aumenta, los valores de objetivo también tienden a subir, lo que podría indicar un ajuste insuficiente del modelo.

El **Number of Estimators** muestra que los valores de objetivo bajos se encuentran más frecuentemente en un rango menor de estimadores. Esto puede indicar que un número menor de estimadores es suficiente para lograr un buen rendimiento y que un número excesivo puede llevar a un sobreajuste, aumentando así los valores de objetivo.

En cuanto a los parámetros de regularización, tanto el **Regularization Alpha (reg_alpha)** como el **Regularization Lambda (reg_lambda)**, se observa que los valores de objetivo más bajos están asociados con valores más bajos de estos parámetros. Esto sugiere que una menor regularización podría ser beneficiosa para el rendimiento del modelo, mientras que una mayor regularización tiende a aumentar los valores de objetivo.

En conclusión, los valores de objetivo más bajos están generalmente asociados con tasas de aprendizaje más bajas, pesos de hijos mínimos más bajos y valores más bajos de los parámetros de regularización (reg_alpha y reg_lambda). Los otros parámetros (max_depth, max_leaves y n_estimators) no muestran una tendencia clara y consistente con respecto a los valores de objetivo, lo que sugiere que su impacto puede depender de la interacción con otros factores.

In [50]:
# Gráfico de importancia de parámetros
plot_param_importances(study)

A partir del gráfico de importancias de hiperparámetros proporcionado, se puede observar que los dos hiperparámetros con mayor relevancia para la optimización de su modelo son **max_leaves** y **learning_rate**.

El hiperparámetro **max_leaves** tiene una importancia de 0.61, lo que lo convierte en el más influyente en el desempeño del modelo. Esto sugiere que el número máximo de hojas en el árbol de decisión tiene un impacto significativo en el valor objetivo del modelo. Por lo tanto, ajustar adecuadamente este parámetro puede resultar crucial para mejorar el rendimiento del modelo.

Por otro lado, **learning_rate** presenta una importancia de 0.31, indicando que la tasa de aprendizaje también juega un papel crucial en la optimización del modelo. Este parámetro, al igual que max_leaves, debe ser ajustado con cuidado, ya que puede tener un efecto considerable en el rendimiento final del modelo.

En comparación, los demás hiperparámetros tienen una importancia mucho menor:

- **reg_lambda**: 0.04
- **reg_alpha**: 0.03
- **n_estimators**: 0.02
- **max_depth**: <0.01
- **min_child_weight**: <0.01

Esto indica que, aunque estos hiperparámetros también contribuyen al rendimiento del modelo, su impacto es significativamente menor que el de max_leaves y learning_rate. Por consiguiente, se recomienda enfocar la mayor parte del esfuerzo de optimización en ajustar estos dos hiperparámetros para mejorar el rendimiento del modelo.

## 6. Síntesis de resultados (0.3)

Finalmente:

1. Genere una tabla resumen del MAE obtenido en los 5 modelos entrenados desde Baseline hasta XGBoost con Constraints, Optuna y Prunning.
2. Compare los resultados de la tabla y responda, ¿qué modelo obtiene el mejor rendimiento?
3. Cargue el mejor modelo, prediga sobre el conjunto de **test** y reporte su MAE.
4. ¿Existen diferencias con respecto a las métricas obtenidas en el conjunto de validación? ¿Porqué puede ocurrir esto?

In [51]:
# Inserte su código acá
mae1 = mean_absolute_error(validation_df['quantity'], pipeline.predict(validation_df))
mae2 = mean_absolute_error(validation_df['quantity'], pipeline_xgb.predict(validation_df))  # XBRegressor por defecto
mae3 = mean_absolute_error(validation_df['quantity'], pipeline_xgb_constrained.predict(validation_df))  # XBRegressor con restricciones
mae4 = mean_absolute_error(validation_df['quantity'], pipeline_optimized.predict(validation_df))  # XBRegressor con optimización
mae5 = mean_absolute_error(validation_df['quantity'], pipeline_optimized_pruning.predict(validation_df))  # XBRegressor con optimización y prunning

mae_df = pd.DataFrame(
    {
        "Modelo": ["DummyRegressor", "XGBRegressor (por defecto)", "XGBRegressor (con restricciones)", "XGBRegressor (con optimización)", "XGBRegressor (con optimization y prunning)"],
        "MAE": [mae1, mae2, mae3, mae4, mae5],
    }
)

mae_df

,Modelo,MAE
0,DummyRegressor,13298.497767
1,XGBRegressor (por defecto),162.195255
2,XGBRegressor (con restricciones),156.243885
3,XGBRegressor (con optimización),134.151960
4,XGBRegressor (con optimization y prunning),134.764592


Al analizar estos valores, se observa que el modelo **XGBRegressor (con optimización)** obtiene el mejor rendimiento con un MAE de 134.151960. Este valor es el más bajo entre todos los modelos evaluados, lo que indica que las predicciones realizadas por este modelo están más cerca de los valores reales en comparación con los otros modelos.

El modelo **XGBRegressor (con optimización y pruning)** también muestra un rendimiento muy bueno, con un MAE de 134.764592, ligeramente superior al del modelo optimizado sin pruning. Ambos modelos optimizados mejoran significativamente el rendimiento en comparación con el modelo **XGBRegressor (por defecto)** y el **XGBRegressor (con restricciones)**, y son mucho más precisos que el **DummyRegressor**.

En conclusión, el modelo **XGBRegressor (con optimización)** es el que obtiene el mejor rendimiento, dado su MAE más bajo de 134.151960, lo que indica una mayor precisión en sus predicciones en comparación con los otros modelos considerados.

In [56]:
mae_test = mean_absolute_error(test_df['quantity'], pipeline_optimized.predict(test_df))
print(f"MAE: {mae_test:.2f}")

MAE: 140.95


Al comparar este resultado con las métricas obtenidas en el conjunto de validación, observamos que el mejor modelo (XGBRegressor con optimización) tenía un MAE de 134.15 en el conjunto de validación. La diferencia entre el MAE del conjunto de validación y el conjunto de test es de aproximadamente 6.8.

Existen varias razones por las cuales esta diferencia puede ocurrir. En primer lugar, la variabilidad en los datos puede influir significativamente. Es posible que el conjunto de test tenga características o distribuciones ligeramente diferentes a las del conjunto de validación, lo que puede afectar el rendimiento del modelo.

Otra razón posible es el sobreajuste. Aunque el modelo se optimizó bien en el conjunto de validación, puede haber ocurrido un ligero sobreajuste a esos datos específicos. Esto se manifiesta cuando el modelo se desempeña un poco peor en datos no vistos, como el conjunto de test.

También es importante considerar la representatividad de las muestras. Si el conjunto de validación no es completamente representativo de todas las posibles variaciones en los datos, mientras que el conjunto de test presenta una distribución más amplia de casos, esto puede explicar la diferencia en el rendimiento.

Finalmente, la aleatoriedad inherente en los datos puede contribuir a esta variabilidad. Incluso con un buen modelo, siempre habrá alguna variabilidad en el rendimiento entre diferentes subconjuntos de datos.

En resumen, aunque el MAE del conjunto de test es ligeramente superior al del conjunto de validación, sigue estando en un rango razonable. Esto sugiere que el modelo generaliza bien, aunque siempre es importante asegurarse de que los datos de validación y test sean lo más representativos posible del problema general para minimizar estas diferencias.

# Conclusión
Eso ha sido todo para el lab de hoy, recuerden que el laboratorio tiene un plazo de entrega de una semana. Cualquier duda del laboratorio, no duden en contactarnos por mail o U-cursos.

<p align="center">
  <img src="https://media.tenor.com/8CT1AXElF_cAAAAC/gojo-satoru.gif">
</p>

<a style='text-decoration:none;line-height:16px;display:flex;color:#5B5B62;padding:10px;justify-content:end;' href='https://deepnote.com?utm_source=created-in-deepnote-cell&projectId=87110296-876e-426f-b91d-aaf681223468' target="_blank">
 </img>
Created in <span style='font-weight:600;margin-left:4px;'>Deepnote</span></a>